# **Step 4: More realistic scenario in production**

<br>

## **Set up scenario**

In [1]:
import pandas as pd

In [2]:
from datetime import datetime

In [3]:
d_dict = {'id': [1,2,3,4,5],
          'address': ['location A','location B','location C','location D','location E'],
          'updated_date': [datetime(2023,1,1),datetime(2023,2,2),datetime(2023,3,3),datetime(2023,4,4),datetime(2023,5,5)]}

In [4]:
d_frame = pd.DataFrame(d_dict)

In [5]:
d_frame

,id,address,updated_date
0,1,location A,2023-01-01
1,2,location B,2023-02-02
2,3,location C,2023-03-03
3,4,location D,2023-04-04
4,5,location E,2023-05-05


**'d_frame' could be from extract and transform process**

<br>

<br>

## **Note**

In [6]:
# 'updated_date' has time component

for i in d_frame.to_records(index=False):
    print(i)

(1, 'location A', '2023-01-01T00:00:00.000000000')
(2, 'location B', '2023-02-02T00:00:00.000000000')
(3, 'location C', '2023-03-03T00:00:00.000000000')
(4, 'location D', '2023-04-04T00:00:00.000000000')
(5, 'location E', '2023-05-05T00:00:00.000000000')


In [7]:
# Do these to remove time component

# Step 1: Form new column without time component
d_frame['updated_date_text'] = d_frame['updated_date'].dt.strftime('%Y-%m-%d')

# Step 2: Drop old column
d_frame.drop(columns=['updated_date'], inplace=True)

# Step 3: Rename new column to old column
d_frame.rename(columns={'updated_date_text': 'updated_date'}, inplace=True)

In [8]:
# 'updated_date' has no time component now

for i in d_frame.to_records(index=False):
    print(i)

(1, 'location A', '2023-01-01')
(2, 'location B', '2023-02-02')
(3, 'location C', '2023-03-03')
(4, 'location D', '2023-04-04')
(5, 'location E', '2023-05-05')


<br>

## **Establish connection and create table**

In [9]:
import sqlite3

In [10]:
conn = sqlite3.connect('mysqlite_database.db')

In [11]:
query_string = """
    CREATE TABLE customer_table (
        id             integer   primary key,
        address        text,
        updated_date   text
    )
    """

In [12]:
conn.execute(query_string)

<br>

## **Insert data**

In [13]:
d_frame['id'] = d_frame['id'].astype('int32')
d_frame['address'] = d_frame['address'].astype('object')
d_frame['updated_date'] = d_frame['updated_date'].astype('object')
print(d_frame.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            5 non-null      int32 
 1   address       5 non-null      object
 2   updated_date  5 non-null      object
dtypes: int32(1), object(2)
memory usage: 232.0+ bytes
None


<br>

**This step needed to ensure matching of data between Python and SQLite**

In [14]:
import numpy as np
sqlite3.register_adapter(np.int32, lambda val: int(val))

<br>

**'data_insert' is format to insert using 'conn.executemany'**

In [15]:
data_insert = list(d_frame.to_records(index=False))

In [16]:
query_string = """
    INSERT OR REPLACE INTO customer_table
        VALUES
            (?,?,?)
    """

In [17]:
conn.executemany(query_string, data_insert)

In [18]:
conn.commit()

In [19]:
from IPython.display import Image 
Image(url='step_04a.png', width=300, height=300)

<br>

## **Insert new data and update existing data**

In [20]:
# id=3 has changed location
# id=6 and id=7 are new

d_dict = {'id': [3,6,7],
          'address': ['location C1','location F','location G'],
          'updated_date': [datetime(2023,3,10),datetime(2023,6,6),datetime(2023,7,7)]}

In [21]:
d_frame = pd.DataFrame(d_dict)

In [22]:
d_frame['updated_date_text'] = d_frame['updated_date'].dt.strftime('%Y-%m-%d')
d_frame.drop(columns=['updated_date'], inplace=True)
d_frame.rename(columns={'updated_date_text': 'updated_date'}, inplace=True)

In [23]:
d_frame['id'] = d_frame['id'].astype('int32')
d_frame['address'] = d_frame['address'].astype('object')
d_frame['updated_date'] = d_frame['updated_date'].astype('object')

In [24]:
data_insert = list(d_frame.to_records(index=False))

In [25]:
query_string = """
    INSERT OR REPLACE INTO customer_table
        VALUES
            (?,?,?)
    """

In [26]:
conn.executemany(query_string, data_insert)

In [27]:
conn.commit()

In [28]:
from IPython.display import Image 
Image(url='step_04b.png', width=300, height=300)

<br>

## **Close connection to database**

In [29]:
conn.close()